In [ ]:
using DataStructures # -> declaración de datos
#some useful reminders --> https://learnxinyminutes.com/docs/julia/
#otro useful reminder --> https://docs.julialang.org/en/v1/base/sort/

# Procesos

Desde un punto de vista computacional, una de las características más significativas de los algoritmos de flujos es que nos obligan a estructurar las interacciones entre varios códigos para resolver el problema conjuntamente.

Aquí veremos un caso, flujo máximo que necesitará el desarrollo de varios bloques que compondrán el conjunto final

## Funcionamiento de caminos mínimos

In [ ]:
type grafoCaminos
    vertices::Int64
    longitud::Array{Int64,2} #longitud[i,j] indica la longitud -> también se puede usar para existencia de arco. Si un arco existe tiene un costo bajo, si no existe, tiene un costo altísimo
end

function generarGrafoCaminos(n::Int64,maxLongitud::Int64)
  a=Array{Int64}(n,n) #matriz longitud
  rand!(a,1:maxLongitud) #aquí se llena la matriz longitud con distancias aleatorias
  return grafoCaminos(n,a)
end


In [ ]:
# Algoritmo de Djikstra
#----------------------------------
function djikstra(grafo::grafoCaminos,origen::Int64) #primero el grafo, segundo qué vértice es el origen
    ∞=typemax(Int64) #el valor infinito
    tr=zeros(Int64,grafo.vertices) # tr -> tr[i] es el último vértice visitado en el camino entre origen e i
    l=Array{Int64}(grafo.vertices) # l -> l[i] es la distancia entre origen y el vértice i 
    fill!(l,∞) #todas las distancias son infinito (no se puede llegar de origen a ninguna parte)
    l[origen]=0 #excepto el vértice origen
    Q=PriorityQueue()
    enqueue!(Q,origen,0) #guardar un vértice en la priority queue
    while isempty(Q)==false #mientras la cola no esté vacía
        v=dequeue!(Q) #voy a intentar mejorar los caminos desde v
        for w in 1:grafo.vertices # mirar los arcos entre[v,w] donde w es cualquier vértice
            if l[w] > (l[v]+grafo.longitud[v,w]) #si el camino actual l[w] es mayor al nuevo camino
                l[w]=l[v]+grafo.longitud[v,w] #cambio longitud
                tr[w]=v #y cambio último vértice
                if haskey(Q,w) #guardo en cola de prioridad
                    Q[w]=l[w]
                else
                    enqueue!(Q,w,l[w])
                end #fin guardo en cola de prioridad
            end
            assert(l[w]>(0-1000000)) #si el valor del camino entre origen y w es muy pequeño, (negativo) va a haber un problema, para de calcular
        end
    end
    return l,tr #devolver el vector de distancias y el vector que se usa para recontruir el camino
end

In [ ]:
# Algoritmo de Bellman Ford
#---------------------------
function bellmanFord(g::grafoCaminos,origen::Int64)
  # paso 1 (inicializamos)
  ∞=typemax(Int64)
  tr=zeros(Int64,g.vertices)
  l=Array{Int64}(g.vertices)
  fill!(l,∞)
  l[origen]=0 #hasta aquí es como Djikstra
  # paso 2 (recurrencia de programación dinámica)
  for i = 1:g.vertices-1 #número de arcos (voy a buscar caminos con 1 arco, 2 arcos, 3 arcos...)
    for v in 1:g.vertices
      for w in 1:g.vertices #dos loops para pasar por todos los arcos -> camino hasta v + arco[v,w]
        if l[w]> (l[v]+g.longitud[v,w]) #si el mejor camino conocido hasta ahora (con i-1 arcos) es peor que camino con i arcos + arco[v,w]
          l[w]=l[v]+g.longitud[v,w] #hay que mejorar
          tr[w]=v
        end
      end
    end
  end
  # paso 3 (búsqueda de un circuito)
  for v in 1:g.vertices
    for w in 1:g.vertices #dos loops para pasar por todos los arcos
      if l[w] > (l[v]+g.longitud[v,w])
        #println("longitud: ",l[w]," vs ",l[v]+g.longitud[v,w]," y v: ",v," w: ",w)
        visited=zeros(Bool,g.vertices)
        while visited[w]==false
          visited[w]=true
          w=tr[w]
        end
        return w,l,tr #indicamos un vértice que forma parte del circuito, la longitud desde origen al resto y el vértice que me permitirá reconstruir el circuito
      end
    end
  end
  return 0,l,tr #salida normal, 0 porque no hay circuitos
end

In [ ]:
unGrafo=generarGrafoCaminos(25,10000)
unGrafo.longitud[13,14]=1; unGrafo.longitud[14,15]=1; unGrafo.longitud[15,16]=1; unGrafo.longitud[16,17]=(-10); 
unGrafo.longitud[17,13]=1; 
conCircuito,longitud,traza=bellmanFord(unGrafo,1)
println("traza: ",traza)
println("vertice conCircuito:",conCircuito)
# Si he encontrado un circuito, imprimirlo
if conCircuito>0
  println("arco: (",traza[conCircuito],",",conCircuito,")")
  w=traza[conCircuito]
  while w != conCircuito
    println("arco: (",traza[w],",",w,")")
    w=traza[w]
  end
end


In [ ]:
longitud,traza=djikstra(unGrafo,1) #si se descomenta salta la "assertion" de circuito negativo

## Funcionamiento flujo máximo

In [ ]:
type grafoFlujos #estructura que necesito para problema de flujos
    vertices::Int64 #los vértices
    capacidad::Array{Int64,2} #capacidad[i,j] indica la capacidad del arco (la capacidad entre parejas de vértices)
    solucion::Array{Int64,2}  #solucion[i,j] indica el flujo del arco (la solución)
end

function generarGrafoFlow(n::Int64,maxCapacidad::Int64,density::Float64) #rutina que genera grafos para flujos de forma aleatoria
  a=Array{Int64}(n,n)
  b=Array{Int64}(n,n)
  rand!(a,1:maxCapacidad)
  for i in 1:n
    a[i,i]=0
    for j in i+1:n
      if rand()>(2*density) #números entre 0 y 1
        a[i,j]=0
        a[j,i]=0
      else # aceptaremos flujo en un único sentido 
        if rand()<0.5
          a[i,j]=0
        else
          a[j,i]=0
        end
      end
    end
  end
  return grafoFlujos(n,a,b)
end

In [ ]:
# Implementación de Ford Fulkerson
#---------------------------------
function fordFulkerson(g::grafoFlujos,origen::Int64,destino::Int64)
  fill!(g.solucion,0) #limpiar la solución
  residual=grafoCaminos(g.vertices,zeros(Int64,g.vertices,g.vertices)) #grafo de soporte
  obj=0
  #println("g.capacidad. ",g.capacidad)
  while true 
    #1. creo el grafo residual
    fill!(residual.longitud,g.vertices+1)
    for i in 1:g.vertices
      for j in 1:g.vertices
        if g.capacidad[i,j]>0 #si tiene capacidad  puede ser que haya arco en sentido directo e inverso
          if g.solucion[i,j]<g.capacidad[i,j] #si no está saturado hay arco en sentido directo
            residual.longitud[i,j]=1 #directo i,j
          end
          if g.solucion[i,j]>0 #si tiene flujo positivo hay arco en sentido inverso
            residual.longitud[j,i]=1 #inverso j,i
          end
        end
      end
    end
    #2. resuelvo Dijkstra
    #println(residual)
    longitud,traza=djikstra(residual,origen)
    println("longitud:",longitud[destino])
    if longitud[destino]<g.vertices #si he podido hacer un camino en el grafo residual entre origen y destino
      extraFlow=typemax(Int32) 
      # primero tenemos que ver el flujo máximo que podemos pasar
      c=destino
      while true #busco el camino y determino su capacidad
        #print("arco entre: ",traza[c]," y ",c)
        if g.capacidad[traza[c],c]>0
          #estoy añadiendo flujo
          extraFlow=min(extraFlow,g.capacidad[traza[c],c]-g.solucion[traza[c],c])
          #println(" capacidad ",g.capacidad[traza[c],c]-g.solucion[traza[c],c]," extra: ",extraFlow)
        else
          #esto eliminando flujo (redirección)
          extraFlow=min(extraFlow,g.solucion[c,traza[c]])
          #println(" backwards ",g.solucion[c,traza[c]]," extra: ",extraFlow)
        end
        c=traza[c]
        if c==origen
          break
        end
      end
      # segundo tenemos que actualizar el flujo de los arcos según flujo máximo
      c=destino
      while true
        #print("cambio en arco ",traza[c],",",c)
        if g.capacidad[traza[c],c]>0
          g.solucion[traza[c],c] += extraFlow
          #println(" added ",extraFlow)
          assert(g.solucion[traza[c],c]<=g.capacidad[traza[c],c])
        else
          g.solucion[c,traza[c]] -= extraFlow
          #println(" removed ",extraFlow)
          assert(g.solucion[traza[c],c]>=0)
        end
        c=traza[c]
        if c==origen
          break
        end
      end
      obj += extraFlow
    else
      #no podemos mejorar la solución
      return obj
    end
    #println("solucion (un paso): ",g.solucion)
  end
  return totalFlow
end

In [ ]:
gEjemplo=grafoFlujos(7,[0 1 3 0 0 0 0 ; 0 0 0 5 4 0 0 ; 0 0 0 3 0 0 0 ; 0 0 0 0 0 0 2 ; 0 0 0 0 0 2 0 ; 0 0 0 0 0 0 3 ; 0 0 0 0 0 0 0],zeros(Int64,7,7))
maxFlow=fordFulkerson(gEjemplo,1,7)
println("maxFlow: ",maxFlow)
println("solucion: ",gEjemplo.solucion)

In [ ]:
gClase=grafoFlujos(8,[0 100 0 0 0 0 0 0; 0 0 60 30 0 0 0 0; 0 0 0 0 20 40 0 0; 0 0 0 0 0 30 0 0; 0 0 0 0 0 0 30 0; 0 0 0 0 0 0 60 0; 0 0 0 0 0 0 0 90; 0 0 0 0 0 0 0 0],zeros(Int64,8,8))
flow=fordFulkerson(gClase,1,8)
println("maxFlow: ",flow)
println("solucion: ",gClase.solucion)

In [ ]:
gFlujos=generarGrafoFlow(10,500,0.45)
maxFlow=fordFulkerson(gFlujos,1,10)
println("maxFlow: ",maxFlow)
println("solucion: ",gFlujos.solucion)

## Funcionamiento flujo máximo a coste mínimo

In [ ]:
type grafoMinCost
    grafoF::grafoFlujos
    coste::Array{Int64,2}  #coste[i,j] indica el coste de pasar una unidad de flujo por el arco
end

function generarGrafoMinCost(n::Int64,maxCapacidad::Int64,maxCost::Int64,density::Float64)
  a=Array{Int64}(n,n)
  b=Array{Int64}(n,n)
  c=Array{Int64}(n,n)
  rand!(a,1:maxCapacidad)
  for i in 1:n
    a[i,i]=0
    for j in i+1:n
      if rand()>(2*density) #números entre 0 y 1
        a[i,j]=0
        a[j,i]=0
      else
        if rand()<0.5
          a[i,j]=0
        else
          a[j,i]=0
        end
      end
    end
  end
  for i in 1:n
    for j in 1:n
      if a[i,j]>0
        c[i,j]=rand(1:maxCost)
      else
        c[i,j]=0
      end
    end
  end
  return grafoMinCost(grafoFlujos(n,a,b),c)
end

In [ ]:
# Implementación
#----------------------------------
function minCostMaxFlow(g::grafoMinCost,origen::Int64,destino::Int64)
  # primer paso. resolver maxflow
  maxFlow=fordFulkerson(g.grafoF,origen,destino)
  coste=0 # determinamos su coste
  for i in 1:g.grafoF.vertices
    for j in 1:g.grafoF.vertices #arcos entre [i,j]
      if g.grafoF.solucion[i,j]>0 #si hay flujo
        coste += g.grafoF.solucion[i,j]*g.coste[i,j] #sumo al coste
      end
    end
  end
  println("maxFlow: ",maxFlow,"\t","coste inicial: ",coste)
  # la idea ahora es la misma que e flujos máximos, pero usamos Bellman Ford para detectar un ciclo de coste negativo 
  # en el grafo
  residual=grafoCaminos(g.grafoF.vertices+1,zeros(Int64,g.grafoF.vertices+1,g.grafoF.vertices+1))
  while true #mientras mejora
    #Primero creo un grafo residual
    fill!(residual.longitud,typemax(Int32)) 
    for i in 1:g.grafoF.vertices
      for j in 1:g.grafoF.vertices
        if g.grafoF.capacidad[i,j]>0  
          if g.grafoF.solucion[i,j]<g.grafoF.capacidad[i,j]
            residual.longitud[i,j]=g.coste[i,j]
          end
          if g.grafoF.solucion[i,j]>0
            residual.longitud[j,i]=0-g.coste[i,j]
          end
        end
      end
    end
    for j in 1:g.grafoF.vertices
        residual.longitud[g.grafoF.vertices+1,j]=0
    end
    #println("residual: ",residual)
    #Luego resuelvo Bellman-Ford
    conCircuito,longitud,traza=bellmanFord(residual,g.grafoF.vertices+1)
    println("conCircuito: ",conCircuito," ",traza)
    #Si hay un circuito
    if conCircuito!=0
      #detecto circuito
      cMin=typemax(Int32)
      w=traza[conCircuito]
      while w != conCircuito
        if residual.longitud[traza[w],w]>0 #arco que añade
          c=g.grafoF.capacidad[traza[w],w]-g.grafoF.solucion[traza[w],w]
        else # arco que reduce
          c=g.grafoF.solucion[w,traza[w]] 
        end
        cMin=min(c,cMin)
        #println(w," ",traza[w]," -> ",residual.longitud[traza[w],w],"  ** ",c)
        w=traza[w]
      end
      if residual.longitud[traza[w],w]>0 # arco que añade
        c=g.grafoF.capacidad[traza[w],w]-g.grafoF.solucion[traza[w],w]
      else # arco que reduce
        c=g.grafoF.solucion[w,traza[w]]
      end
      cMin=min(c,cMin)
      #println(w," ",traza[w]," -> ",residual.longitud[traza[w],w]," -- ",c)
      # alterar el flujo en el circuito
      w=traza[conCircuito]
      while w != conCircuito
        if residual.longitud[traza[w],w]>0
          g.grafoF.solucion[traza[w],w] += cMin
          assert(g.grafoF.solucion[traza[w],w]<=g.grafoF.capacidad[traza[w],w])
          coste += cMin*g.coste[traza[w],w]
        else
          g.grafoF.solucion[w,traza[w]] -= cMin
          assert(g.grafoF.solucion[w,traza[w]]<=g.grafoF.capacidad[w,traza[w]])
          coste -= cMin*g.coste[w,traza[w]]
        end
        w=traza[w]
      end
      if residual.longitud[traza[w],w]>0
        g.grafoF.solucion[traza[w],w] += cMin
        assert(g.grafoF.solucion[traza[w],w]<=g.grafoF.capacidad[traza[w],w])
        coste += cMin*g.coste[traza[w],w]
      else # arco que reduce
        g.grafoF.solucion[w,traza[w]] -= cMin
        assert(g.grafoF.solucion[w,traza[w]]>=0)
        coste -= cMin*g.coste[w,traza[w]]
      end
      println("coste: ",coste)
    else
      return coste
    end
  end
end

In [ ]:
gMinCostEjemplo=grafoMinCost(grafoFlujos(4,[0 3 2 0; 0 0 1 0; 0 0 0 2; 0 0 0 0],zeros(Int64,4,4)),[0 10 50 0; 0 0 30 0; 0 0 0 20; 0 0 0 0])
coste=minCostMaxFlow(gMinCostEjemplo,1,4)
println("coste final: ",coste)

In [ ]:
gMinCost=generarGrafoMinCost(50,50,1000,0.45)
coste=minCostMaxFlow(gMinCost,1,50)